## Canadian Hospital Readmittance Challenge 

This notebook is made as a part of the Machine Learning (AI-511) project. It has been made by the following students - 

1. Siddharth Kothari (IMT2021019)
2. Sankalp Kothari (IMT2021028)
3. M Srinivasan (IMT2021058)


In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import xgboost as xgb
import optuna
from math import floor
from sklearn.metrics import accuracy_score,confusion_matrix,ConfusionMatrixDisplay
import re
pd.options.display.max_rows = 4000

In [6]:
df = pd.read_csv('../canadian-hospital-re-admittance-challenge/train.csv')
test_df = pd.read_csv('../canadian-hospital-re-admittance-challenge/test.csv')
df

,enc_id,patient_id,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmission_id
0,88346340,2488608,Caucasian,Male,[60-70),NaN,1,2,6,3,...,No,Steady,No,No,No,No,No,Ch,Yes,2
1,92001408,52133202,Caucasian,Male,[70-80),[100-125),2,6,1,7,...,No,No,No,No,No,No,No,No,Yes,1
2,169424316,40945509,Caucasian,Female,[70-80),NaN,3,2,1,7,...,No,Up,No,No,No,No,No,Ch,Yes,1
3,272987082,38850777,Caucasian,Female,[50-60),NaN,1,1,7,1,...,No,No,No,No,No,No,No,No,Yes,2
4,150600612,72738225,Caucasian,Female,[80-90),NaN,1,6,7,6,...,No,Down,No,No,No,No,No,Ch,Yes,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71231,198619164,85063725,Caucasian,Male,[70-80),NaN,1,1,7,6,...,No,No,No,No,No,No,No,No,Yes,1
71232,177404100,86244345,Caucasian,Male,[90-100),NaN,1,3,7,5,...,No,No,No,No,No,No,No,No,No,2
71233,50905206,5131368,Caucasian,Male,[70-80),NaN,3,6,1,6,...,No,Steady,No,No,No,No,No,Ch,Yes,2
71234,216431502,85969035,Hispanic,Male,[50-60),NaN,1,1,4,4,...,No,Steady,No,No,No,No,No,No,Yes,2


### EDA and Preprocessing

In [7]:
df.columns

Index(['enc_id', 'patient_id', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmission_id'],
      dtype='object')

#### Null Removal

We first see the percentage of null values in each of the columns, to see which columns we can immediatelty drop.

In [8]:
percent_missing = df.isnull().sum() * 100 / len(df)

percentages_df = pd.DataFrame({
    'percent_missing': percent_missing
})

percentages_df

,percent_missing
enc_id,0.000000
patient_id,0.000000
race,2.275535
gender,0.000000
age,0.000000
weight,96.841485
admission_type_id,0.000000
discharge_disposition_id,0.000000
admission_source_id,0.000000
time_in_hospital,0.000000


We see that the following columns have a very large number of null values - 
1. Weight (96.84)
2. A1Cresult (83.32)
3. max_glu_serum (94.77)


We thus try to see the distribution of the values in these columns to see whether any useful info can be gained.

In [9]:
df['max_glu_serum'].value_counts()

max_glu_serum
Norm    1790
>200    1034
>300     897
Name: count, dtype: int64

In [10]:
df['A1Cresult'].value_counts()

A1Cresult
>8      5715
Norm    3476
>7      2689
Name: count, dtype: int64

In [11]:
df['weight'].value_counts()

weight
[75-100)     944
[50-75)      643
[100-125)    421
[125-150)    103
[25-50)       69
[0-25)        34
[150-175)     27
[175-200)      7
>200           2
Name: count, dtype: int64

##### Dropping Columns

Since no useful info can be gained from these columns, we drop them altogether.

We also drop columns which do not seem to be relevant to the readmission of a patient, such as payer code.

In [12]:
df.drop(columns=['weight','payer_code','max_glu_serum','A1Cresult'], inplace=True)
test_df.drop(columns=['weight','payer_code','max_glu_serum','A1Cresult'],inplace=True)

df

,enc_id,patient_id,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmission_id
0,88346340,2488608,Caucasian,Male,[60-70),1,2,6,3,Family/GeneralPractice,...,No,Steady,No,No,No,No,No,Ch,Yes,2
1,92001408,52133202,Caucasian,Male,[70-80),2,6,1,7,NaN,...,No,No,No,No,No,No,No,No,Yes,1
2,169424316,40945509,Caucasian,Female,[70-80),3,2,1,7,NaN,...,No,Up,No,No,No,No,No,Ch,Yes,1
3,272987082,38850777,Caucasian,Female,[50-60),1,1,7,1,NaN,...,No,No,No,No,No,No,No,No,Yes,2
4,150600612,72738225,Caucasian,Female,[80-90),1,6,7,6,NaN,...,No,Down,No,No,No,No,No,Ch,Yes,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71231,198619164,85063725,Caucasian,Male,[70-80),1,1,7,6,NaN,...,No,No,No,No,No,No,No,No,Yes,1
71232,177404100,86244345,Caucasian,Male,[90-100),1,3,7,5,NaN,...,No,No,No,No,No,No,No,No,No,2
71233,50905206,5131368,Caucasian,Male,[70-80),3,6,1,6,Cardiology,...,No,Steady,No,No,No,No,No,Ch,Yes,2
71234,216431502,85969035,Hispanic,Male,[50-60),1,1,4,4,InternalMedicine,...,No,Steady,No,No,No,No,No,No,Yes,2


##### Medical Specialty

For the medical specialty column, we do not directly drop it, as some specialists' patients may have more chance of being readmitted than others'. Hence to tackle null values there, we impute the null values with a new value - "No-admitting-Physician".

This assumption is supported by the image below

![Medical Specialty](../images/Medical%20Specialty.png)

In [13]:
df["medical_specialty"].fillna("No-Admitting-Physician", inplace=True)
test_df["medical_specialty"].fillna("No-Admitting-Physician", inplace=True)

In [14]:
percent_missing = df.isnull().sum() * 100 / len(df)

percentages_df = pd.DataFrame({
    'percent_missing': percent_missing
})

percentages_df

,percent_missing
enc_id,0.000000
patient_id,0.000000
race,2.275535
gender,0.000000
age,0.000000
admission_type_id,0.000000
discharge_disposition_id,0.000000
admission_source_id,0.000000
time_in_hospital,0.000000
medical_specialty,0.000000


##### Diag Groupings

The columns for diag_1, diag_2, diag_3 have a lot of categorical values, which we have grouped referring to the following website listing the groupings of ICD codes.

https://en.wikipedia.org/wiki/List_of_ICD-9_codes

We decided to drop the diag_1 column, and keep the other two, based on the following plots made in Tableau, which show the variation of the data with the columns. Readmission_id did not vary much with diag_1, hence we dropped it.

![Diag_1_group](../images/Diag_1_group.png)

![Diag_2_group](../images/Diag_2_group.png)

![Diag_3_group](../images/Diag_3_group.png)


In [15]:
diag_grouping_dict = {
    0 : [0],
    1 : range(1,140),
    2 : range(140,240),
    3 : range(240,280),
    4 : range(280,290),
    5 : range(290,320), 
    6 : range(320,390),
    7 : range(390,460),
    8 : range(460,520),
    9 : range(520,580),
    10: range(580,630),
    11: range(630,680),
    12: range(680,710),
    13: range(710,740),
    14: range(740,760),
    15: range(760-780),
    16: range(780,800),
    17: range(800,1000)
}

def diag_convert2(row):
    if str(row['diag_2'])[0] in ['E','V']:
        return 18
    else:
        for j in diag_grouping_dict.keys():
            if floor(float(row['diag_2'])) in diag_grouping_dict[j]:
                return j
            

def diag_convert3(row):
    if str(row['diag_3'])[0] in ['E','V']:
        return 18
    else:
        for j in diag_grouping_dict.keys():
            if floor(float(row['diag_3'])) in diag_grouping_dict[j]:
                return j

In [16]:
df.fillna("0", inplace=True)
test_df.fillna("0", inplace=True)

df.drop(columns=['diag_1'], inplace=True)
test_df.drop(columns=['diag_1'], inplace=True)

new_col = df.apply(diag_convert2, axis=1)
df.insert(loc = len(df.columns)-1, column = 'diag_2_new', value=new_col)
df.drop(columns=['diag_2'], inplace=True)

new_col = test_df.apply(diag_convert2, axis=1)
test_df.insert(loc = len(test_df.columns), column = 'diag_2_new', value=new_col)
test_df.drop(columns=['diag_2'], inplace=True)

new_col = df.apply(diag_convert3, axis=1)
df.insert(loc = len(df.columns)-1, column = 'diag_3_new', value=new_col)
df.drop(columns=['diag_3'], inplace=True)

new_col = test_df.apply(diag_convert3, axis=1)
test_df.insert(loc = len(test_df.columns), column = 'diag_3_new', value=new_col)
test_df.drop(columns=['diag_3'], inplace=True)

Another possibility we had considered was to drop diag_1, diag_2, diag_3 altogether, and instead replace it with a new column, which captures some of the information of the columns. However this did not bring about better results, so we dropped it.

In [17]:
# df.loc[df['diag_1'].notnull(), 'diag_1'] = 4
# df.loc[df['diag_2'].notnull(), 'diag_2'] = 2
# df.loc[df['diag_3'].notnull(), 'diag_3'] = 1

# df['diag_1'].fillna(0,inplace=True)
# df['diag_2'].fillna(0,inplace=True)
# df['diag_3'].fillna(0,inplace=True)

# test_df.loc[test_df['diag_1'].notnull(), 'diag_1'] = 4
# test_df.loc[test_df['diag_2'].notnull(), 'diag_2'] = 2
# test_df.loc[test_df['diag_3'].notnull(), 'diag_3'] = 1

# test_df['diag_1'].fillna(0,inplace=True)
# test_df['diag_2'].fillna(0,inplace=True)
# test_df['diag_3'].fillna(0,inplace=True)

# df.loc[:,'diag_1':'diag_3']

In [18]:
# new_col = df['diag_1']+df['diag_2']+df['diag_3']
# df.insert(loc = len(df.columns)-1, column = 'Number_of_Diagnosis', value=new_col)

# new_col = test_df['diag_1']+test_df['diag_2']+test_df['diag_3']
# test_df.insert(loc = len(test_df.columns), column = 'Number_of_Diagnosis', value=new_col)

In [19]:
# df.drop(columns=['diag_1','diag_2','diag_3'], inplace=True)
# test_df.drop(columns=['diag_1','diag_2','diag_3'], inplace=True)

##### Race and Gender

Finally, using the plots below, we realised that the race and gender of the person did not affect the values of readmission_id observed, hence we felt it better to drop these two columns.

Our claims are supported by the charts below.


![Gender](../images/Gender.png)

![Race](../images/Race.png)

In [20]:
df.drop(columns=['race','gender'], inplace=True)
test_df.drop(columns=['race','gender'], inplace=True)
df

,enc_id,patient_id,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,...,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,diag_2_new,diag_3_new,readmission_id
0,88346340,2488608,[60-70),1,2,6,3,Family/GeneralPractice,54,3,...,No,No,No,No,No,Ch,Yes,7,7,2
1,92001408,52133202,[70-80),2,6,1,7,No-Admitting-Physician,50,6,...,No,No,No,No,No,No,Yes,7,7,1
2,169424316,40945509,[70-80),3,2,1,7,No-Admitting-Physician,49,4,...,No,No,No,No,No,Ch,Yes,8,17,1
3,272987082,38850777,[50-60),1,1,7,1,No-Admitting-Physician,1,1,...,No,No,No,No,No,No,Yes,10,3,2
4,150600612,72738225,[80-90),1,6,7,6,No-Admitting-Physician,58,2,...,No,No,No,No,No,Ch,Yes,13,12,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71231,198619164,85063725,[70-80),1,1,7,6,No-Admitting-Physician,61,0,...,No,No,No,No,No,No,Yes,3,7,1
71232,177404100,86244345,[90-100),1,3,7,5,No-Admitting-Physician,36,0,...,No,No,No,No,No,No,No,8,8,2
71233,50905206,5131368,[70-80),3,6,1,6,Cardiology,62,6,...,No,No,No,No,No,Ch,Yes,4,3,2
71234,216431502,85969035,[50-60),1,1,4,4,InternalMedicine,29,1,...,No,No,No,No,No,No,Yes,9,4,2


#### Engineering new features

We also made some new features which we thought would better capture the data. 

1. Admission_type_id_new - here we grouped the various values in the columns as given in admission_grouping_dict to reduce the number of categories.
2. Discharge_type_id_new - similar as previous. We grouped them based on which categories broadly fell into the same umbrella according to our understanding. 
3. Admission_source_id_new - similar as previous.

The mapping of the number to the category is as follows -

##### Admission type id
| admission_type_id | description |
| ----------------- | ----------- |
| 1 | Emergency |
| 2 | Urgent |
| 3 | Elective |
| 4 | Newborn |
| 5 | Not Available |
| 6 | NULL |
| 7 | Trauma Center |
| 8 | Not Mapped |

##### Discharge Disposition id

| discharge_disposition_id | description |
| ------------------------ | ----------- |
| 1 | Discharged to home |
| 2 | Discharged/transferred to another short term hospital |
| 3 | Discharged/transferred to SNF(Skilled Nursing Facility). |
| 4 | Discharged/transferred to ICF(Intermediate Care Facility). |
| 5 | Discharged/transferred to another type of inpatient care institution |
| 6 | Discharged/transferred to home with home health service |
| 7 | Left AMA (American Medical Association) |
| 8 | Discharged/transferred to home under care of Home IV provider |
| 9 | Admitted as an inpatient to this hospital |
| 10 | Neonate discharged to another hospital for neonatal aftercare |
| 11 | Expired |
| 12 | Still patient or expected to return for outpatient services |
| 13 | Hospice(Nursing Home for ill/old) / home |
| 14 | Hospice / medical facility |
| 15 | Discharged/transferred within this institution to Medicare approved swing bed |
| 16 | Discharged/transferred/referred another institution for outpatient services |
| 17 | Discharged/transferred/referred to this institution for outpatient services |
| 18 | NULL |
| 19 | "Expired at home. Medicaid only, hospice." |
| 20 | "Expired in a medical facility. Medicaid only, hospice." |
| 21 | "Expired, place unknown. Medicaid only, hospice." |
| 22 | Discharged/transferred to another rehab fac including rehab units of a hospital. |
| 23 | Discharged/transferred to a long term care hospital. |
| 24 | Discharged/transferred to a nursing facility certified under Medicaid but not certified under Medicare. |
| 25 | Not Mapped |
| 26 | Unknown/Invalid |
| 30 | Discharged/transferred to another Type of Health Care Institution not Defined Elsewhere |
| 27 | Discharged/transferred to a federal health care facility. |
| 28 | Discharged/transferred/referred to a psychiatric hospital of psychiatric distinct part unit of a hospital |
| 29 | Discharged/transferred to a Critical Access Hospital (CAH). |


##### Admission source id
| admission_source_id | description |
| ------------------- | ----------- |
| 1  | Physician Referral |
| 2  | Clinic Referral |
| 3  | HMO Referral |
| 4  | Transfer from a hospital |
| 5  | Transfer from a Skilled Nursing Facility (SNF) | 
| 6  | Transfer from another health care facility | 
| 7  | Emergency Room | 
| 8  | Court/Law Enforcement |
| 9  | Not Available |
| 10 | Transfer from critial access hospital |
| 11 | Normal Delivery |
| 12 | Premature Delivery |
| 13 | Sick Baby |
| 14 | Extramural Birth |
| 15 | Not Available |
| 17 | NULL |
| 18 | Transfer From Another Home Health Agency |
| 19 | Readmission to Same Home Health Agency |
| 20 | Not Mapped |
| 21 | Unknown/Invalid |
| 22 | Transfer from hospital inpt/same fac reslt in a sep claim |
| 23 | Born inside this hospital |
| 24 | Born outside this hospital |
| 25 | Transfer from Ambulatory Surgery Center |
| 26 | Transfer from Hospice |

In [21]:
admission_grouping_dict = {
    1 : [1],
    2 : [2],
    3 : [3],
    4 : [4],
    5 : [5,6,8], 
    6 : [7]
}

def admission_group(row):
    for j in admission_grouping_dict.keys():
        if row['admission_type_id'] in admission_grouping_dict[j]:
            return j

new_col = df.apply(admission_group, axis=1)
df.insert(loc = len(df.columns)-1, column = 'admission_type_id_new', value=new_col)
df.drop(columns=['admission_type_id'], inplace=True)

new_col = test_df.apply(admission_group, axis=1)
test_df.insert(loc = len(test_df.columns), column = 'admission_type_id_new', value=new_col)
test_df.drop(columns=['admission_type_id'], inplace=True)

df['admission_type_id_new'].value_counts()

admission_type_id_new
1    37831
3    13188
2    12979
5     7223
4        8
6        7
Name: count, dtype: int64

In [22]:
discharge_grouping_dict = {
    1 : [11,19,20,21],
    2 : [18,25,26],
    3 : [7],
    4 : [1,6,8,13,14],
    5 : [2,3,4,5,10,16,22,23,24,30,27,28,29],
    6 : [9,12,15,17]
}

def discharge_group(row):
    for j in discharge_grouping_dict.keys():
        if row['discharge_disposition_id'] in discharge_grouping_dict[j]:
            return j
        
new_col = df.apply(discharge_group, axis=1)
df.insert(loc = len(df.columns)-1, column = 'discharge_type_id_new', value=new_col)
df.drop(columns=['discharge_disposition_id'], inplace=True)

new_col = test_df.apply(discharge_group, axis=1)
test_df.insert(loc = len(test_df.columns), column = 'discharge_type_id_new', value=new_col)
test_df.drop(columns=['discharge_disposition_id'], inplace=True)

df['discharge_type_id_new'].value_counts()

discharge_type_id_new
4    51763
5    14489
2     3319
1     1160
3      444
6       61
Name: count, dtype: int64

In [23]:
source_grouping_dict = {
    1 : [4,5,6,10,18,22,25,26],
    2 : [1,2,3],
    3 : [11,12,13,14],
    4 : [9,15,17,20,21],
    5 : [7],
    6 : [8]
}

def source_group(row):
    for j in source_grouping_dict.keys():
        if row['admission_source_id'] in source_grouping_dict[j]:
            return j

new_col = df.apply(source_group, axis=1)
df.insert(loc = len(df.columns)-1, column = 'admission_source_id_new', value=new_col)
df.drop(columns=['admission_source_id'], inplace=True)

new_col = test_df.apply(source_group, axis=1)
test_df.insert(loc = len(test_df.columns), column = 'admission_source_id_new', value=new_col)
test_df.drop(columns=['admission_source_id'], inplace=True)

df['admission_source_id_new'].value_counts()

admission_source_id_new
5    40268
2    21610
4     4877
1     4465
6       11
3        5
Name: count, dtype: int64

4. Service_utilization - we felt that the total number of visits by a person (emergency + inpatient + outpatient) can be a good indicator of whether the person will be readmitted or not. Hence we created this new feature. 

In [24]:
new_col = df['number_outpatient'] + df['number_emergency'] + df['number_inpatient']
df.insert(loc = len(df.columns)-1, column = 'service_utilization', value=new_col)

new_col = test_df['number_outpatient'] + test_df['number_emergency'] + test_df['number_inpatient']
test_df.insert(loc = len(test_df.columns), column = 'service_utilization', value=new_col)

df

,enc_id,patient_id,age,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,...,metformin-pioglitazone,change,diabetesMed,diag_2_new,diag_3_new,admission_type_id_new,discharge_type_id_new,admission_source_id_new,service_utilization,readmission_id
0,88346340,2488608,[60-70),3,Family/GeneralPractice,54,3,10,0,0,...,No,Ch,Yes,7,7,1,5,1,0,2
1,92001408,52133202,[70-80),7,No-Admitting-Physician,50,6,35,2,1,...,No,No,Yes,7,7,2,4,2,3,1
2,169424316,40945509,[70-80),7,No-Admitting-Physician,49,4,20,2,0,...,No,Ch,Yes,8,17,3,5,2,6,1
3,272987082,38850777,[50-60),1,No-Admitting-Physician,1,1,15,0,0,...,No,No,Yes,10,3,1,4,5,0,2
4,150600612,72738225,[80-90),6,No-Admitting-Physician,58,2,26,1,0,...,No,Ch,Yes,13,12,1,4,5,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71231,198619164,85063725,[70-80),6,No-Admitting-Physician,61,0,14,0,0,...,No,No,Yes,3,7,1,4,5,0,1
71232,177404100,86244345,[90-100),5,No-Admitting-Physician,36,0,12,2,0,...,No,No,No,8,8,1,5,5,2,2
71233,50905206,5131368,[70-80),6,Cardiology,62,6,28,0,0,...,No,Ch,Yes,4,3,3,4,2,0,2
71234,216431502,85969035,[50-60),4,InternalMedicine,29,1,11,0,0,...,No,No,Yes,9,4,1,4,1,0,2


5. Age - age is made as a categorical variable here, and we felt it should be numeric. Hence for each range (eg. 40-50) we replace the value with the median of the range, to make it numeric.

In [25]:
def age_converter(row):
    if row['age'] == '[0-10)':
        return 5
    elif row['age'] == '[10-20)':
        return 15
    elif row['age'] == '[20-30)':
        return 25
    elif row['age'] == '[30-40)':
        return 35
    elif row['age'] == '[40-50)':
        return 45
    elif row['age'] == '[50-60)':
        return 55
    elif row['age'] == '[60-70)':
        return 65
    elif row['age'] == '[70-80)':
        return 75
    elif row['age'] == '[80-90)':
        return 85
    elif row['age'] == '[90-100)':
        return 95

new_col = df.apply(age_converter, axis=1)
df['age'] = new_col

new_col = test_df.apply(age_converter, axis=1)
test_df['age'] = new_col

df['age'].value_counts()

age
75    18179
65    15801
55    12080
85    12037
45     6785
35     2650
95     1940
25     1165
15      495
5       104
Name: count, dtype: int64

##### Patient visit count

We felt the number of entries of admittance of a patient (indicated by the number of records of a given patient id) can be a useful measure of whether that person has been readmitted. Hence we count the number of records of the patient and make a new column "patient_id_new" for the same. We then drop enc_id and patient_id as those are anyways not useful features, and as such can't tell anything about the readmittance of a patient.

In [26]:
patient = df['patient_id'].value_counts()
patient = patient.add(test_df['patient_id'].value_counts(),fill_value=0)
patient = patient.astype('int64')

def id_convertor(row):
    return patient[row['patient_id']]

new_col = df.apply(id_convertor, axis=1)
df.insert(loc = len(df.columns)-1, column = 'patient_id_new', value=new_col)
df.drop(columns=['patient_id'],inplace=True)
df.drop(columns=['enc_id'], inplace=True)

new_col = test_df.apply(id_convertor, axis=1)
test_df.insert(loc = len(test_df.columns), column = 'patient_id_new', value=new_col)
test_df.drop(columns=['patient_id'], inplace=True)
test_df.drop(columns=['enc_id'], inplace=True)

##### Drug manipulations

We realised that for a lot of patients, many of the drugs mentioned are either not administered, or their dosage has not been changed. Hence we first count the number for each of the drugs.

In [27]:
for col in df.loc[:,'metformin':'diabetesMed']:
    med_groups = df.groupby(by=[col])
    print(med_groups.count().iloc[:, 0])

metformin
Down        392
No        57223
Steady    12885
Up          736
Name: age, dtype: int64
repaglinide
Down         33
No        70145
Steady      980
Up           78
Name: age, dtype: int64
nateglinide
Down          8
No        70770
Steady      444
Up           14
Name: age, dtype: int64
chlorpropamide
No        71170
Steady       62
Up            4
Name: age, dtype: int64
glimepiride
Down        131
No        67558
Steady     3308
Up          239
Name: age, dtype: int64
acetohexamide
No        71235
Steady        1
Name: age, dtype: int64
glipizide
Down        376
No        62301
Steady     8011
Up          548
Name: age, dtype: int64
glyburide
Down        389
No        63713
Steady     6577
Up          557
Name: age, dtype: int64
tolbutamide
No        71221
Steady       15
Name: age, dtype: int64
pioglitazone
Down         83
No        66074
Steady     4910
Up          169
Name: age, dtype: int64
rosiglitazone
Down         67
No        66740
Steady     4303
Up          126
Na

We then see that the following drugs have a very small number of records where the dosage of a drug has been changed, and as such it won't contribute much to the model. Hence we remove the following drugs - 
1. chlorpropamide
2. tolbutamide
3. miglitol
4. acarbose
5. tolazamide
6. acetohexamide
7. troglitazone
8. examide
9. citoglipton
10. glipizide-metformin
11. glimepiride-pioglitazone
12. metformin-rosiglitazone
13. metformin-pioglitazone
14. glyburide-metformin

In [28]:
df.drop(columns=['chlorpropamide', 'tolbutamide', 'miglitol', 'acarbose', 'tolazamide', 'acetohexamide', 'troglitazone', 'examide', 'citoglipton', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'glyburide-metformin'], inplace=True)
test_df.drop(columns=['chlorpropamide', 'tolbutamide', 'miglitol', 'acarbose', 'tolazamide', 'acetohexamide', 'troglitazone', 'examide', 'citoglipton', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'glyburide-metformin'], inplace=True)
df.columns.size

29

For the remaining drugs, we then count the number of "changes" of drug dosages, i.e. for a person, how many of the dosages either went up or down. We make this as a new feature - "changes", and then drop all the remaining drug columns.

In [29]:
def count_changes(row):
    count =0
    for col in ['metformin','repaglinide','nateglinide','glimepiride','glipizide','glyburide','pioglitazone','rosiglitazone','insulin']:
        if(row[col]=='Up' or row[col]=='Down'):
            count+=1
    return count

new_col = df.apply(count_changes, axis=1)
df.insert(loc = len(df.columns)-1, column = 'changes', value=new_col)

new_col = test_df.apply(count_changes, axis=1)
test_df.insert(loc = len(test_df.columns), column = 'changes', value=new_col)

In [30]:
df.drop(columns=['metformin','repaglinide','nateglinide','glimepiride','glipizide','glyburide','pioglitazone','rosiglitazone','insulin','change'], inplace=True)
test_df.drop(columns=['metformin','repaglinide','nateglinide','glimepiride','glipizide','glyburide','pioglitazone','rosiglitazone','insulin','change'], inplace=True)

df.columns.size

20

Based on the following plots, we also considered dropping the following columns, as they didn't seem to affect the distribution of readmission_id.

1. num_procedures
2. time_in_hospital

![num_procedures](../images/num_procedures.png)

![time_in_hospital](../images/time_in_hospital.png)

But that led to a decrease in accuracy, so we didn't end up doing this.

In [31]:
# df.drop(columns=['num_procedures','time_in_hospital'])
# test_df.drop(columns=['num_procedures','time_in_hospital'])

#### Data Visualization 

We plotted the following plots to get a better idea as to how the data affects readmission_id.
 
1. Scatterplot matrix between the various numeric features. The code for it has been commented out below, as it takes considerable amount of time to run. The plot has been downloaded and displayed below.
2. Covariance plot between the various numeric features and readmission_id. The code for the same is available below, and it is also commented out.  
3. Stacked bar charts for categorical variables, similar to the ones shown previously. They have been attached as well.


![Splom](../images/scatterplot_matrix.png)

![Covariance](../images/covariance.png)

![Admission_type_id](../images/Admission_type_id.png)

![Discharge_disposition_id](../images/Discharge_disposition_id.png)

![Admission_source_id](../images/Admission_source_id.png)

The other plots can be found in the images directory. We weren't able to infer much from them, and hence they weren't included in the report.

In [32]:
# plt.figure(figsize=(20,20))
# corr = df.loc[:, ["time_in_hospital","num_lab_procedures","num_procedures","num_medications","number_outpatient","number_emergency","number_inpatient","number_diagnoses","changes", "readmission_id"]].corr()
# sns.heatmap(corr,annot=True)

In [33]:
# index_vals = df['readmission_id'].astype('category').cat.codes

# dimensions = []

# cols = ['num_lab_procedures','num_procedures','num_medications','time_in_hospital','service_utilization','number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'changes']

# for i in cols:
#     d = dict(label=i, values=df[i])
#     dimensions.append(d)

# fig = go.Figure(data=go.Splom(
#      dimensions=dimensions,
#      diagonal_visible=False,
#      text=df['readmission_id'],
#      marker=dict(color=index_vals,
#                line_color='white', line_width=0.5)
# ))

# fig.update_layout(
#     title='Readmission id',
#     dragmode='select',
#     width=1500,
#     height=1500,
#     hovermode='closest',
# )

# fig.show()


#### Model fitting 

We tried out various combinations of models for the data. Before that, we first carried out a few steps - 

1. We standard scaled the numerical columns.
2. We one-hot encoded all the categorical columns with 3 or more categories. This led to certain columns which were there in the test data but not in the training data (as some categories appeared only in the test data), and some others which were only available in the training data.
3. To handle this, we do the following - add all columns from the train data not present in the test data with all values as zero, while the columns in the test data not available in the test data are simply dropped. We then sort the columns based on name to ensure correct ordering in training and testing data.
4. We then do a train test split with the test size as 0.2, and proceed to fit the models.
5. We also tried introducing polynomial features, but it decreased the performance of the model, so we removed it.

In [34]:
df.columns

Index(['age', 'time_in_hospital', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'number_diagnoses',
       'diabetesMed', 'diag_2_new', 'diag_3_new', 'admission_type_id_new',
       'discharge_type_id_new', 'admission_source_id_new',
       'service_utilization', 'patient_id_new', 'changes', 'readmission_id'],
      dtype='object')

In [35]:
test_df.columns

Index(['age', 'time_in_hospital', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'number_diagnoses',
       'diabetesMed', 'diag_2_new', 'diag_3_new', 'admission_type_id_new',
       'discharge_type_id_new', 'admission_source_id_new',
       'service_utilization', 'patient_id_new', 'changes'],
      dtype='object')

In [36]:
input = df.loc[:, "age":"changes"]
labels = df.loc[:, "readmission_id"]
input.columns

Index(['age', 'time_in_hospital', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'number_diagnoses',
       'diabetesMed', 'diag_2_new', 'diag_3_new', 'admission_type_id_new',
       'discharge_type_id_new', 'admission_source_id_new',
       'service_utilization', 'patient_id_new', 'changes'],
      dtype='object')

In [37]:
input_encoded = pd.get_dummies(input, columns=['medical_specialty','admission_type_id_new',
                    'discharge_type_id_new','admission_source_id_new','diag_2_new','diag_3_new'])

input_encoded['diabetesMed'] = np.where(input_encoded['diabetesMed']=='Yes',1,0)

print(input_encoded.columns)

Index(['age', 'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'diabetesMed',
       ...
       'diag_3_new_8', 'diag_3_new_9', 'diag_3_new_10', 'diag_3_new_11',
       'diag_3_new_12', 'diag_3_new_13', 'diag_3_new_14', 'diag_3_new_16',
       'diag_3_new_17', 'diag_3_new_18'],
      dtype='object', length=136)


In [38]:
test_encoded = pd.get_dummies(test_df, columns=['medical_specialty','admission_type_id_new', 
                    'discharge_type_id_new','admission_source_id_new','diag_2_new','diag_3_new'])

test_encoded['diabetesMed'] = np.where(test_encoded['diabetesMed']=='Yes',1,0)


print(test_encoded.columns)

Index(['age', 'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'diabetesMed',
       ...
       'diag_3_new_8', 'diag_3_new_9', 'diag_3_new_10', 'diag_3_new_11',
       'diag_3_new_12', 'diag_3_new_13', 'diag_3_new_14', 'diag_3_new_16',
       'diag_3_new_17', 'diag_3_new_18'],
      dtype='object', length=130)


In [39]:
for i in input_encoded.columns:
    if i not in test_encoded.columns:
       test_encoded[i] = 0

for i in test_encoded.columns:
    if i not in input_encoded.columns:
       test_encoded.drop(columns=[i], inplace=True)

input_encoded.sort_index(axis=1, inplace=True)
test_encoded.sort_index(axis=1, inplace=True)

In [40]:
print(input_encoded.columns)
print(test_encoded.columns)

Index(['admission_source_id_new_1', 'admission_source_id_new_2',
       'admission_source_id_new_3', 'admission_source_id_new_4',
       'admission_source_id_new_5', 'admission_source_id_new_6',
       'admission_type_id_new_1', 'admission_type_id_new_2',
       'admission_type_id_new_3', 'admission_type_id_new_4',
       ...
       'num_lab_procedures', 'num_medications', 'num_procedures',
       'number_diagnoses', 'number_emergency', 'number_inpatient',
       'number_outpatient', 'patient_id_new', 'service_utilization',
       'time_in_hospital'],
      dtype='object', length=136)
Index(['admission_source_id_new_1', 'admission_source_id_new_2',
       'admission_source_id_new_3', 'admission_source_id_new_4',
       'admission_source_id_new_5', 'admission_source_id_new_6',
       'admission_type_id_new_1', 'admission_type_id_new_2',
       'admission_type_id_new_3', 'admission_type_id_new_4',
       ...
       'num_lab_procedures', 'num_medications', 'num_procedures',
       'number

We remove the standart scaling of data to main the original data as it is.

In [41]:
# scaler = StandardScaler()
# input_encoded[['patient_id_new','age','num_lab_procedures','number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'changes','num_procedures','num_medications','time_in_hospital']] = scaler.fit_transform(input_encoded[['patient_id_new','age','num_lab_procedures','number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'changes','num_procedures','num_medications','time_in_hospital']].to_numpy())
# test_encoded[['patient_id_new','age','num_lab_procedures','number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'changes','num_procedures','num_medications','time_in_hospital']] = scaler.fit_transform(test_encoded[['patient_id_new','age','num_lab_procedures','number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'changes','num_procedures','num_medications','time_in_hospital']].to_numpy())

In [42]:
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('transformer', PolynomialFeatures(degree=2, include_bias=False), ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']),
#     ],
#     remainder='passthrough'
# )

# input_encoded = preprocessor.fit_transform(input_encoded)
# test_encoded = preprocessor.transform(test_encoded)

input_encoded['readmission_id'] = labels
input_encoded

,admission_source_id_new_1,admission_source_id_new_2,admission_source_id_new_3,admission_source_id_new_4,admission_source_id_new_5,admission_source_id_new_6,admission_type_id_new_1,admission_type_id_new_2,admission_type_id_new_3,admission_type_id_new_4,...,num_medications,num_procedures,number_diagnoses,number_emergency,number_inpatient,number_outpatient,patient_id_new,service_utilization,time_in_hospital,readmission_id
0,True,False,False,False,False,False,True,False,False,False,...,10,3,5,0,0,0,1,0,3,2
1,False,True,False,False,False,False,False,True,False,False,...,35,6,9,1,0,2,2,3,7,1
2,False,True,False,False,False,False,False,False,True,False,...,20,4,9,0,4,2,8,6,7,1
3,False,False,False,False,True,False,True,False,False,False,...,15,1,8,0,0,0,1,0,1,2
4,False,False,False,False,True,False,True,False,False,False,...,26,2,9,0,0,1,1,1,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71231,False,False,False,False,True,False,True,False,False,False,...,14,0,7,0,0,0,3,0,6,1
71232,False,False,False,False,True,False,True,False,False,False,...,12,0,9,0,0,2,1,2,5,2
71233,False,True,False,False,False,False,False,False,True,False,...,28,6,9,0,0,0,1,0,6,2
71234,True,False,False,False,False,False,True,False,False,False,...,11,1,9,0,0,0,1,0,4,2


Once all the pre-processing is done then we put the data into:
1) preprocessed_data.csv
2) test_data.csv 


In [43]:
input_encoded.to_csv("processed_data.csv", index=False)
test_encoded.to_csv("test_data.csv", index=False)